In [0]:
!pip install tensorflow-gpu==1.15.2
!pip install keras==2.3

!/usr/local/cuda/bin/nvcc --version
import tensorflow as tf
tf.__version__

!tar -xzvf drive/My\ Drive/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

import tensorflow as tf
tf.test.gpu_device_name()

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

     |████████████████████████████████| 411.0MB 36kB/s 
     |████████████████████████████████| 3.8MB 34.9MB/s 
     |████████████████████████████████| 512kB 40.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=15c7574820d09850e2243b8218859b416f42369c210151923ec3ed61b30fc75c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer

from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
df = pd.read_csv('/content/drive/My Drive/SEM 6/SE project/Dataset/train1.csv')
test = pd.read_csv('/content/drive/My Drive/SEM 6/SE project/Dataset/test1.csv')

df.head()

In [0]:
def preprocess(df):
    df['bully'] = np.where( (df.toxic == 1) | (df.severe_toxic==1) | (df.obscene == 1) | (df.threat == 1) | (df.insult == 1) | (df.identity_hate == 1), 1, 0)
    
    df = df.drop(columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']).rename(columns={'comment_text':'comment'})
    
    df = df.replace('\n',' ', regex=True)
    
    df["comment"] = df['comment'].str.replace('[^\w\s]','')
    
    df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    stop = stopwords.words('english')
    df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))  

    return df

In [0]:
df = preprocess(df)

In [0]:
df.head()

In [0]:
df[df['bully'] == 1].shape

In [0]:
print(df[ df['bully'] == 1].size)
print(df[ df['bully'] == 0].size)

In [0]:
import re
second_data = pd.read_csv('/content/train2.csv')
second_data.drop(columns=['Date'],inplace=True)
second_data.dropna(inplace=True)
second_data.rename(columns={'Insult':'bully','Comment':'comment'},inplace=True)
print(second_data.shape)

third_data = pd.read_csv('/content/test2.csv')
third_data.drop(columns=['Date'],inplace=True)
third_data.dropna(inplace=True)
third_data.rename(columns={'Insult':'bully','Comment':'comment'},inplace=True)
print(third_data.shape)

df = pd.concat([df,second_data[['comment','bully']],third_data[['comment','bully']]])
df.shape

In [0]:
X = df.comment
Y = np.array(df.bully)
Y = Y.reshape(-1,1)

In [0]:
X_train,Y_train=X,Y
max_words = 10000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

In [0]:
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding


def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
    
model.fit(sequences_matrix,Y_train,batch_size=512,epochs=10,
              validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])



In [0]:
def testSentence(text):

    X_test=text
    print(X_test)
    test_sequences=""
    test_sequences = tok.texts_to_sequences(X_test)
    print(test_sequences)
    test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
    print(test_sequences_matrix)
    ans=model.predict(test_sequences_matrix,batch_size=None,verbose=0,steps=None)
    st=""
    st=st.join(str(ans[0][0]))
    print(st)

testSentence('you are dumb')
print()
testSentence('i am happy for you')
print()
testSentence('you will go to hell')

you are dumb
0.09002742

i am happy for you
0.056275725

you will go to hell
0.09002742


In [0]:
model.save('cbds.h5')

In [0]:
from tensorflow.keras.models import load_model
model_1 = load_model('cbds.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model_1.save('cbds_tf.h5')

In [0]:

print(type(model))
pickl = {
    'model': model_1,
    'tokenizer': tok
}

pickle.dump( pickl, open( 'models_tf_keras' + ".p", "wb" ) )

In [0]:
with open('models.p', 'rb') as pickled:
       data = pickle.load(pickled)
model1 = data['model']
tokenizer = data['tokenizer']

# from keras.models import load_model
model1

In [0]:
X_test=[input("Enter ")]
print(X_test)
test_sequences=""
test_sequences = tok.texts_to_sequences(X_test)
print(test_sequences)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
# print(test_sequences_matrix)
ans=model1.predict(test_sequences_matrix,batch_size=None,verbose=0,steps=None)
st=""
st=st.join(str(ans[0][0]))
print(st)
ans